In [1]:
from database.market import Market
from database.sec import SEC
import pandas as pd
from tqdm import tqdm
from datetime import datetime, timedelta, timezone
pd.options.mode.chained_assignment = None

In [2]:
market = Market()
sec = SEC()

In [3]:
market.connect()
sp5 = market.retrieve("sp500")
prices = market.retrieve("alpha_prices")
market.disconnect()

In [4]:
prices["date"] = prices["date"] = pd.to_datetime(prices["date"])

In [5]:
analysis = []
for ticker in tqdm(sp5["Symbol"].unique()):
    try:
        ticker_data = prices[prices["ticker"]==ticker]
        ticker_data.sort_values("date",ascending=False)
        ticker_data["prev"] = ticker_data["date"].shift(-1)
        ticker_data["diff"] = ticker_data["prev"] - ticker_data["date"]
        ticker_data["diff_days"] = [x.days for x in ticker_data["diff"]]
        max_diff = ticker_data["diff_days"].max()
        strange_dates = ticker_data[ticker_data["diff_days"]> 5]
        for row in strange_dates.iterrows():
            sd = row[1]["date"]
            ro = row[1]["prev"]
            diff = row[1]["diff_days"]
            analysis.append({"ticker":ticker,
                             "diff":diff,
                             "occurence_date":sd,
                             "return_date":ro})
    except Exception as e:
        print(str(e))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [02:05<00:00,  4.01it/s]


In [6]:
a = pd.DataFrame(analysis)

In [7]:
a[a["occurence_date"]==a["occurence_date"].max()]

,ticker,diff,occurence_date,return_date
111,DISCK,6.0,2022-04-08,2022-04-14


In [10]:
a["return_date"].unique()

array(['2001-09-17T00:00:00.000000000', '2022-04-14T00:00:00.000000000',
       '2021-06-02T00:00:00.000000000'], dtype='datetime64[ns]')

In [45]:
sec.connect()
quarterlies = []
for row in tqdm(list(sp5.iterrows())):
    try:
        cik = row[1]["CIK"]
        ticker = row[1]["Symbol"]
        funds = sec.retrieve_filing_data(cik)
        funds["filed"] = [datetime.strptime(str(x),"%Y%m%d").replace(tzinfo=timezone.utc) if "-" not in str(x) else \
                         datetime.strptime(str(x).split(" ")[0],"%Y-%m-%d").replace(tzinfor=timezone.utc) for x in funds["filed"]]
        funds["quarter"] = [x.quarter for x in funds["filed"]]
        funds["year"] = [x.year for x in funds["filed"]]
        funds["ticker"] = ticker
        start = int(funds["year"].min())
        end = int(funds["year"].max())
        for year in range(start,end):
            for quarter in range(1,5):
                if year == 2021 and quarter == 4 or (year == 2009 and quarter < 3):
                    continue
                else:
                    filing = funds[(funds["year"]==year) & (funds["quarter"]==quarter)]
                    if filing.index.size < 1:
                        quarterlies.append({
                            "ticker":ticker,
                            "year":year,
                            "quarter":quarter
                        })
                    else:
                        continue
    except Exception as e:
        print(str(e))
sec.disconnect()

 38%|█████████████████████████████████████████████████▌                                                                               | 194/505 [00:56<01:31,  3.40it/s]

'filed'


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 505/505 [02:27<00:00,  3.42it/s]


In [46]:
qa = pd.DataFrame(quarterlies)

In [52]:
qa[(qa["year"]>2012) & (qa["ticker"]=="GOOGL")]

,ticker,year,quarter
64,GOOGL,2015,1
65,GOOGL,2015,2
66,GOOGL,2015,3
